In [29]:
import os
import numpy
import matplotlib.pyplot as pyplot
from itertools import groupby 
import random
import shutil
%matplotlib inline

In [48]:
import seaborn as sns
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix

Using TensorFlow backend.


In [46]:
def structureData(source_data, train_perc):
    images = []

    for (dirpath, dirnames, filenames) in os.walk(source_data):
        images.extend(filenames)

    data_path = './data'
    if not os.path.exists(data_path):
        os.makedirs(data_path)

    util_func = lambda x: x[0] 
    temp = sorted(images, key = util_func) 
    res = [list(ele) for i, ele in groupby(temp, util_func)]

    train_path = os.path.join(data_path, 'train')
    os.makedirs(train_path)
    test_path = os.path.join(data_path, 'test')
    os.makedirs(test_path)

    for i in range(len(res)):
        perc = 0.7
        z_0 = res[i]
        train_len = int(len(z_0) * 0.7)
        random.shuffle(z_0)
        train_p = z_0[:train_len]
        test_p = z_0[train_len:]
        tr_path = os.path.join(train_path, str(i))
        os.makedirs(tr_path)
        te_path = os.path.join(test_path, str(i))
        os.makedirs(te_path)
        for name in train_p:
            shutil.copy(os.path.join(source_data, name), tr_path)
        for name in test_p:
            shutil.copy(os.path.join(source_data, name), te_path)

In [47]:
structureData('./dataset', 0.7)

In [50]:
datagen = ImageDataGenerator(
        rescale=1./255,
        zoom_range=0.2,
        rotation_range=10,
        horizontal_flip=True,
        vertical_flip=False)


In [52]:
train_generator = datagen.flow_from_directory(
    'data/train',
    target_size=(256, 256),
    color_mode = 'grayscale',
    batch_size=32,
    class_mode='categorical')

Found 236 images belonging to 7 classes.


In [53]:
test_generator = datagen.flow_from_directory(
    'data/test',
    target_size=(256, 256),
    color_mode = 'grayscale',
    batch_size=32,
    class_mode='categorical')

Found 105 images belonging to 7 classes.
